In [119]:
import os, re
import numpy as np
import tensorflow as tf

In [120]:
import glob


txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [121]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() #문장 양끝 공백 제고
    sentence = '<start> ' + sentence + ' <end>' #문장 양끝에 <start>와 <end> 추가
    return sentence

In [122]:
corpus = []

for sentence in raw_corpus:
        
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence) > 15: continue #15개 이상 제외
        
        
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
corpus[:9]



['<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> she tied you <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> in every word <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> and even though <end>']

In [123]:

def tokenize(corpus):
    # 단어장크기 12000단어
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"# 포함되지 못한 단어는 '<unk>'로바꿈
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')   
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)


[[  2 162   3 ...   0   0   0]
 [  2 162   3 ...   0   0   0]
 [  2  56 881 ...   0   0   0]
 ...
 [  2 216 554 ...   0   0   0]
 [  2  60  17 ...   0   0   0]
 [  2  36  21 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fda703c0790>


In [124]:
print(tensor[ :10, :13]) #뒷쪽에 패딩된 0이 많이 보인다

[[  2 162   3   0   0   0   0   0   0   0   0   0   0]
 [  2 162   3   0   0   0   0   0   0   0   0   0   0]
 [  2  56 881   7   3   0   0   0   0   0   0   0   0]
 [  2 162   3   0   0   0   0   0   0   0   0   0   0]
 [  2 162   3   0   0   0   0   0   0   0   0   0   0]
 [  2  23 280 457   3   0   0   0   0   0   0   0   0]
 [  2 162   3   0   0   0   0   0   0   0   0   0   0]
 [  2 162   3   0   0   0   0   0   0   0   0   0   0]
 [  2  18 351 563   3   0   0   0   0   0   0   0   0]
 [  2 162   3   0   0   0   0   0   0   0   0   0   0]]


In [125]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 30: break  #2번 인덱스가 바로 <start> 단어사전 30개 꺼내보기
        
        

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : .
7 : you
8 : oh
9 : it
10 : me
11 : !
12 : a
13 : the
14 : yeah
15 : my
16 : t
17 : s
18 : and
19 : to
20 : love
21 : on
22 : m
23 : in
24 : baby
25 : no
26 : we
27 : so
28 : ?
29 : chorus
30 : don


In [126]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:] 
print(src_input.shape)


(14120, 12)


In [127]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state=20)

In [128]:
print(enc_train.shape)
print(enc_val.shape)
print(dec_train.shape)
print(dec_val.shape)

(11296, 12)
(2824, 12)
(11296, 12)
(2824, 12)


In [129]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024   
hidden_size = 2048  
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


In [130]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val, dec_val))


Epoch 1/10
353/353 [==============================] - 57s 108ms/step - loss: 1.6137 - val_loss: 1.4074
Epoch 2/10
353/353 [==============================] - 37s 106ms/step - loss: 1.2876 - val_loss: 1.2978
Epoch 3/10
353/353 [==============================] - 37s 105ms/step - loss: 1.1478 - val_loss: 1.2353
Epoch 4/10
353/353 [==============================] - 37s 106ms/step - loss: 1.0412 - val_loss: 1.1881
Epoch 5/10
353/353 [==============================] - 37s 106ms/step - loss: 0.9737 - val_loss: 1.1652
Epoch 6/10
353/353 [==============================] - 37s 106ms/step - loss: 0.9044 - val_loss: 1.1499
Epoch 7/10
353/353 [==============================] - 37s 106ms/step - loss: 0.8579 - val_loss: 1.1428
Epoch 8/10
353/353 [==============================] - 38s 106ms/step - loss: 0.8233 - val_loss: 1.1422
Epoch 9/10
353/353 [==============================] - 37s 106ms/step - loss: 0.7963 - val_loss: 1.1444
Epoch 10/10
353/353 [==============================] - 37s 105ms/step - l

In [131]:
def generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated


In [134]:

# generate_text(model, tokenizer, init_sentence="<start> he")
test_sen = generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
print(test_sen)


<start> i love you <end> 


#### 회고 : 인공지능이 작사를 한다는 것을 생각해 본 적이 없었서 매우 신선했던 내용이었다.
    노드의 예시를 최대한 참고하여 작성하였고 큰 프로세스를 이해하는데 의의가 있다고 스스로 생각하며 작성하였다. 
    여기에 들어간 함수를 직접 만들라고 하면 절대 못했을 것 같다.
    train_test_split은 이제 좀 익숙해 진 것 같다. 
    random state와 embedding size, hidden size 등 파라미터 셋팅은 아직 감이 없어서 찍기로 여러값을 시도해보았다. 